In [1]:
#pip install numpy pandas sklearn joblib

In [2]:
import json # will be needed for saving preprocessing details
import numpy as np # for data manipulation
import pandas as pd # for data manipulation
from sklearn.model_selection import train_test_split # will be used for data split
from sklearn.preprocessing import LabelEncoder # for preprocessing
import joblib # for saving algorithm and preprocessing objects

In [3]:
import statistics
from scipy import stats
from dmba import plotDecisionTree, classificationSummary, regressionSummary
from imblearn.over_sampling import SMOTE
from collections import Counter
from array import array
from sklearn import preprocessing, svm , metrics , tree
from sklearn.model_selection import train_test_split , cross_val_score , GridSearchCV
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

In [4]:
# load dataset
df = pd.read_csv('https://raw.githubusercontent.com/R7amza/project1/R7amza-patch-2/11111.csv', skipinitialspace=True)
df.set_index('Organization_Name', inplace=True)

In [5]:
df = df.drop(columns = ['Description', 'Funding_Status', 'Industries', 'Downloads_Last_30_Days', 'Number_of_Apps'])
df = df.query('Founded_Date >2014')
df["Bounce_Rate"] = df["Bounce_Rate"].str.rstrip("%").astype(float)
df["Monthly_Visits_Growth"] = df["Monthly_Visits_Growth"].str.rstrip("%").astype(float)

In [6]:
x_cols = [c for c in df.columns if c != 'Exit_status']
# set input matrix and target column
X = df[x_cols]
y = df['Exit_status']
# show first rows of data
df.head()

,Estimated_Revenue_Range,Total_Funding_Amount,Headquarters_Location,Founded_Date,Number_of_Founders,Funding_Stage,Number_Funding_Rounds,Number_of_Investors,Industry_Groups,Sector_Size,Number_of_Employees,Visit_Duration,Bounce_Rate,Monthly_Visits,Monthly_Visits_Growth,Exit_status
Organization_Name,,,,,,,,,,,,,,,,
Nana,5500000.0,78959999.0,Riyad,2020,2.0,Undisclosed,6.0,12.0,Food Delivery,42000000000,376,561.0,58.0,80887.0,-30.0,0
Retailo Technologies,NaN,45000000.0,Riyad,2020,3.0,Series A,4.0,13.0,E-Commerce,13000000000,751,652.0,66.0,22598.0,1357.0,0
Lean Technologies,5500000.0,36500000.0,Riyad,2019,3.0,Series A,4.0,13.0,Financial Services,13000000000,31,102.0,87.0,6677.0,-45.0,0
Sary,300000000.0,111957478.0,Riyad,2018,2.0,Series C,4.0,12.0,E-Commerce,13000000000,376,2567.0,40.0,25286.0,258.0,0
Tamara,NaN,116000000.0,Riyad,2020,3.0,Series A,4.0,9.0,Financial Services,13000000000,76,347.0,37.0,143987.0,74.0,0


In [7]:
# data split train / test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state=9999)

In [8]:
cont = ['Estimated_Revenue_Range', 'Total_Funding_Amount',
       'Founded_Date','Number_of_Founders', 
       'Number_Funding_Rounds', 'Number_of_Investors', 'Sector_Size', 
        'Number_of_Employees','Visit_Duration','Bounce_Rate', 'Monthly_Visits', 
        'Monthly_Visits_Growth']

cate = ['Headquarters_Location','Funding_Stage','Industry_Groups']

In [9]:
train_median = X_train[cont].median()
X_train[cont] = X_train[cont].fillna(train_median)
print(train_median)

Estimated_Revenue_Range    5.500000e+06
Total_Funding_Amount       1.199680e+06
Founded_Date               2.017000e+03
Number_of_Founders         2.000000e+00
Number_Funding_Rounds      1.000000e+00
Number_of_Investors        2.000000e+00
Sector_Size                1.300000e+10
Number_of_Employees        3.100000e+01
Visit_Duration             2.910000e+02
Bounce_Rate                5.000000e+01
Monthly_Visits             3.107000e+03
Monthly_Visits_Growth      3.500000e+00
dtype: float64


In [10]:
train_mode = dict(X_train[cate].mode().iloc[0])
X_train[cate] = X_train[cate].fillna(train_mode)
print(train_mode)

{'Headquarters_Location': 'Riyadh ', 'Funding_Stage': 'Seed', 'Industry_Groups': 'E-Commerce'}


In [11]:
# Unifying Headquarters Location names
dict_start = {'Headquarters_Location': {'Riyad ':'Riyadh',
                                        'Riyad':'Riyadh',
                                        'Riyadh ':'Riyadh',
                                        'Jeddah ':'Jeddah',
                                        'Jedda':'Jeddah',
                                        'Sharqiyah':'Ash Sharqiyah',
                                        'Sharqiyah ':'Ash Sharqiyah',
                                        'Ash Sharqiyah ':'Ash Sharqiyah'}}
X_train.replace(dict_start, inplace=True)

In [12]:
#Classifying Funding Stage in to three stages
dict_start2 = {'Funding_Stage': {'Angel ': 'Early Stage', 
                                 'Angel': 'Early Stage',
                                 'Early Stage ': 'Early Stage',
                                 'Pre-Seed ' : 'Early Stage', 
                                 'Pre-Seed' : 'Early Stage',
                                 'Early Stage ': 'Early Stage',
                                 'Seed ': 'Early Stage', 
                                 'Seed': 'Early Stage', 
                                 'Early Stage ': 'Early Stage',
                                 'Series A ': 'Mid stage', 
                                 'Series A': 'Mid stage',
                                 'Mid stage ': 'Mid stage',
                                 'Series B ': 'Mid Stage',
                                 'Series B': 'Mid stage',
                                 'Sereis B': 'Mid stage',
                                 'Mid stage ': 'Mid stage',
                                 'Series C ': 'Late Stage',
                                 'Series C': 'Late stage',
                                 'Sereis C': 'Late stage',
                                 'Late stage ': 'Late stage'}}
X_train.replace(dict_start2, inplace=True)

In [13]:
#To replace 'Undisclosed' category by mode.
X_train['Funding_Stage'] = X_train['Funding_Stage'].replace({'Undisclosed': np.NaN})
X_train['Funding_Stage'] = X_train['Funding_Stage'].fillna(X_train['Funding_Stage'].value_counts().index[0])

In [14]:
#Unifying Industry groups
dict_start3 = {'Industry_Groups': {'Food Delivery  ': 'Food &Beverage ', 
                                   'Food Delivery ': 'Food &Beverage ',   
                                   'FinTech ' : 'Financial Services', 
                                   'FinTech': 'Financial Services', 
                                   'FintTech' : 'Financial Services',
                                   'Consultancy  ': 'Financial Services',
                                   'Financial Services ' : 'Financial Services',
                                   'Computer Vision ': 'Artificial Intelligence',
                                   'Computer Vision': 'Artificial Intelligence',
                                   'Data Visualization ': 'Artificial Intelligence', 
                                   'Data Visualization': 'Artificial Intelligence', 
                                   'Data Analysis': 'Artificial Intelligence',
                                   'Blockchain ': 'Artificial Intelligence', 
                                   'Blockchain': 'Artificial Intelligence',
                                   'Business Intelligence': 'Artificial Intelligence',
                                   'Artificial Intelligenc' : 'Artificial Intelligence',
                                   'Insurtech' : 'Artificial Intelligence',
                                   'Recruitment ': 'Human Resources',
                                   'Recruitment': 'Human Resources',
                                   'Logistics ':'Travel&Toursim',
                                   'Logistics':'Travel&Toursim',
                                   'Transportation' : 'Travel&Toursim',
                                   'Travel&Toursim ': 'Travel&Toursim',
                                   'Travel&Tourism ': 'Travel&Toursim',
                                   'Food&Beverage ' : 'FMCG',
                                   'Food &Beverage ' : 'FMCG',
                                   'Consumer Services': 'FMCG', 
                                   'Healthtech' : 'HealthTech',
                                   'HealthTech ' : 'HealthTech',
                                   'Financial services ': 'Financial Services',
                                   'Financial Services ': 'Financial Services', 
                                    'Real Estate ': 'Real Estate'}}
X_train.replace(dict_start3, inplace=True)

In [15]:
def cap_data(df):
    for col in df.columns:
        print("capping the ",col)
        if (((df[col].dtype)=='float64') | ((df[col].dtype)=='int64')):
            outliers = stats.zscore(df[col])
            df[col][outliers < -3] = statistics.median(df[col])
            df[col][outliers >  3] = statistics.median(df[col])
        else:
            df[col]=df[col]
    return df

X_train=cap_data(X_train)

capping the  Estimated_Revenue_Range
capping the  Total_Funding_Amount
capping the  Headquarters_Location
capping the  Founded_Date
capping the  Number_of_Founders
capping the  Funding_Stage
capping the  Number_Funding_Rounds
capping the  Number_of_Investors
capping the  Industry_Groups
capping the  Sector_Size
capping the  Number_of_Employees
capping the  Visit_Duration
capping the  Bounce_Rate
capping the  Monthly_Visits
capping the  Monthly_Visits_Growth


In [16]:
# convert categoricals
encoders = {}
for column in ['Headquarters_Location', 'Funding_Stage', 'Industry_Groups']:
    categorical_convert = LabelEncoder()
    X_train[column] = categorical_convert.fit_transform(X_train[column])
    encoders[column] = categorical_convert

In [17]:
# Scale the data
PredictorScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X_train)

# Generating the standardized values of X
X_train=PredictorScalerFit.transform(X_train)

In [18]:
# Oversample training data
sm = SMOTE(random_state = 2)
X_train, y_train = sm.fit_resample(X_train, y_train.ravel())

In [19]:
#XGBoost
XGB = XGBClassifier(max_depth=2, learning_rate=0.01, n_estimators=200, objective='binary:logistic', booster='gbtree')
XGB = XGB.fit(X_train, y_train)

In [20]:
# save preprocessing objects and RF algorithm

joblib.dump(train_mode, "./train_mode.joblib", compress=True)
joblib.dump(train_median, "./train_median.joblib", compress=True)
joblib.dump(dict_start, "./dict_start.joblib", compress=True)
joblib.dump(dict_start2, "./dict_start2.joblib", compress=True)
joblib.dump(dict_start3, "./dict_start3.joblib", compress=True)
joblib.dump(encoders, "./encoders.joblib", compress=True)
joblib.dump(X_train, "./X_train.joblib", compress=True)
joblib.dump(XGB, "./xgboost.joblib", compress=True)

['./xgboost.joblib']